## Stream features using Signals

This notebook creates a new feature view using the SDK that will be computed using stream processing.

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Stream processing/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

### Define a new feature

This block creates a single feature definition including the logic how it should be calculated (it's filters and aggregation).

The feature calculates the number of add to cart ecommerce events.

In [3]:
from snowplow_signals_sdk import (
    Feature,
    FilterCombinator,
    FilterCondition,
)

products_added_to_cart_feature = Feature(
    name="products_added_to_cart",
    scope="session",
    dtype="STRING_LIST",
    events=[
        "iglu:com.snowplowanalytics.snowplow.ecommerce/snowplow_ecommerce_action/jsonschema/1-0-2"
    ],
    type="unique_list",
    property="contexts_com_snowplowanalytics_snowplow_ecommerce_product_1_0_0.name",
    filter=FilterCombinator(
        combinator="and",
        condition=[
            FilterCondition(
                property="event.type", operator="equals", value="add_to_cart"
            ),
        ],
    ),
)

### Wrapping the feature in a feature view

All features need to be included in feature views that can be considered as "tables" of features.

Feature views are immutable and versioned.

In [4]:
from snowplow_signals_sdk import FeatureView, session_entity

feature_view = FeatureView(
    name="my_ecommerce_features",
    version=1,
    entities=[
        session_entity,
    ],
    features=[
        products_added_to_cart_feature,
    ],
)

### Applying the feature view to Signals

The following block pushes the feature view definition to the Signals API and makes it available for processing.

In [6]:
from snowplow_signals_sdk import SignalsStore

sp_signals = SignalsStore()
sp_signals.apply([feature_view])

ApplyResponse(status='applied')

### Retrieving data

One can fetch the latest feature values for a particular session from the feature view as follow.

In [15]:
response = sp_signals.get_online_features(
    features=[feature_view],
    entity="c5b91072-06b3-4aa7-a558-8e11dcc09993",
    entity_type_id="session",
)

response.to_dataframe()

,domain_sessionid,products_added_to_cart
0,c5b91072-06b3-4aa7-a558-8e11dcc09993,None
